In [11]:
x <- read.csv("Intervals_092014_012016.csv",colClasses = c("character","numeric","numeric","character"),header=TRUE)
#library(data.table,lib.loc="../Rpackages")

ERROR: Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, : scan() expected 'a real', got 'START_READ'


Warning message in file(file, "rt"):
"cannot open file 'gunzip -c Intervals_092014_012016.csv.gz |': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [1]:
library(data.table)
full = c();

#for(i in 0:1){
#    x <- fread("gunzip -c Intervals_092014_012016.csv.gz | head -1000000",
#           #colClasses=c("character","double","double","character","double"),
#           header=TRUE,sep=",",na.strings="")
#    full= rbind(full, x)
#}
summary(full)

Length  Class   Mode 
     0   NULL   NULL 

In [5]:
# need t
full = c();
x <- fread("Intervals_092014_012016.csv",
#           colClasses=c("character","double","double","character","double"),
           header=TRUE,sep=",",na.strings="",nrows = 55000000);
full = rbind(full,x);
summary(full)

Read 55000000 rows and 5 (of 5) columns from 5.247 GB file in 00:00:43


 MR_MDVC_NUMBER       START_READ       END_READ        INTERVAL_TIME     
 Length:55000000    Min.   :    0   Min.   :    0.08   Length:55000000   
 Class :character   1st Qu.: 6106   1st Qu.: 6117.30   Class :character  
 Mode  :character   Median : 9761   Median : 9778.07   Mode  :character  
                    Mean   :11071   Mean   :11089.86                     
                    3rd Qu.:14405   3rd Qu.:14428.21                     
                    Max.   :99938   Max.   :99937.71                     
                    NA's   :1327                                         
 INTERVAL_READ   
 Min.   : 0.000  
 1st Qu.: 0.064  
 Median : 0.128  
 Mean   : 0.197  
 3rd Qu.: 0.239  
 Max.   :34.394  
 NA's   :19308   

In [15]:
x = x[x[,1] < 1,1]
summary(x)

ERROR: Error in `[.data.table`(x, x[, 1] < 1, 1): i is invalid type (matrix). Perhaps in future a 2 column matrix could return a list of elements of DT (in the spirit of A[B] in FAQ 2.14). Please let datatable-help know if you'd like this, or add your comments to FR #657.


In [6]:
str(x)

Classes 'data.table' and 'data.frame':	55000000 obs. of  5 variables:
 $ MR_MDVC_NUMBER: chr  "123053022" "123053022" "123053022" "123053022" ...
 $ START_READ    : num  8486 8486 8486 8486 8486 ...
 $ END_READ      : num  8498 8498 8498 8498 8498 ...
 $ INTERVAL_TIME : chr  "01-SEP-2014 00:15:00" "01-SEP-2014 00:30:00" "01-SEP-2014 00:45:00" "01-SEP-2014 01:00:00" ...
 $ INTERVAL_READ : num  0.148 0.101 0.141 0.231 0.262 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [ ]:
x <- na.omit(x)
x <- scale(x)

In [ ]:

table(x)
table(x[,1])

In [7]:
table(table(x[,1]))



 6340  6816 10176 11524 13536 13628 15460 17860 18332 19964 23900 23996 24288 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
26496 26880 29088 29184 30350 31776 32064 32160 39488 54279 64112 64127 64128 
    1     1    92     1     1     1  1601     1     1     1     1     1     6 

In [15]:
good = names(table(x[,1]))[table(x[,1])==32046];
str(good)

 chr "125132704"


In [16]:
mm=match(as.matrix(x[,1]),good)
str(mm)

 int [1:55281440] NA NA NA NA NA NA NA NA NA NA ...


In [17]:
x1 = x[!is.na(mm),]

In [18]:
dim(x1)

[1] 32046     5

In [ ]:
res = c();
for (i in 1:1725){
    res = cbind(res, as.matrix(x[((i-1)*32046+1):(i*32046),5]))
}
res = t(res)
str(res)
print("done")

In [ ]:
d <- dist(res, method = "euclidean") # distance matrix
fit <- hclust(d, method="ward") 
plot(fit) # display dendogram

In [ ]:
matplot(t(res[1:3,]),type="l")

In [8]:
#get 5 highest samples
tot = tapply(x$INTERVAL_READ, x$MR_MDVC_NUMBER, sum, na.rm=T)
qtot = quantile(tot,0:10/10);

o = order(-tot);
top = names(tot[o][1:5]);
x_top <- x[match(x$MR_MDVC_NUMBER, top, nomatch=0)>0,]

mid = names(tot[tot<qtot[6]&tot>=qtot[5]])[1:5];
x_mid <- x[match(x$MR_MDVC_NUMBER, mid, nomatch=0)>0,]

bot = names(tot[tot<qtot[3]&tot>=qtot[2]])[1:5];
x_bot <- x[match(x$MR_MDVC_NUMBER, bot, nomatch=0)>0,]

In [13]:
my_plot <- function(tmp_df, i, q){
    name <- paste(i,"_",q,"_full",".jpeg",sep="")
    jpeg(name)
    par(mfrow=c(4,2))
    plot(filter(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,c(1,2,3,4,5,4,3,2,1)/(25)),main=i)
    acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,lag=24*4,main=i,na.action=na.pass)
    acf(tmp_df[tmp_df$MR_MDVC_NUMBER==i,]$INTERVAL_READ,type="partial",lag=24*4,main=i,na.action=na.pass)
    acf(diff(tmp_df$INTERVAL_READ),main=i,na.action=na.pass)
    dev.off()
}

In [14]:
unique(x_bot$MR_MDVC_NUMBER)
for(i in unique(x_bot$MR_MDVC_NUMBER)){
    my_plot(x_bot,i,"bot_big")
    print(i)
}
unique(x_top$MR_MDVC_NUMBER)
for(i in unique(x_top$MR_MDVC_NUMBER)){
    my_plot(x_top,i,"top_big")
    print(i)
}
unique(x_mid$MR_MDVC_NUMBER)
for(i in unique(x_mid$MR_MDVC_NUMBER)){
    my_plot(x_mid,i,"mid_big")
    print(i)
}

[1] "123053025" "123053106" "123053137" "123053169" "123053053"

[1] "123053025"
[1] "123053106"
[1] "123053137"
[1] "123053169"
[1] "123053053"


[1] "123053036" "123053040" "123053916" "123054013" "123054329"

[1] "123053036"
[1] "123053040"
[1] "123053916"
[1] "123054013"
[1] "123054329"


[1] "123053028" "123053035" "123053089" "123053092" "123053103"

[1] "123053028"
[1] "123053035"
[1] "123053089"
[1] "123053092"
[1] "123053103"
